In [4]:
import numpy as np
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )
y["train"] = np.load("./train_label_save.npy")
X["dev"] = np.load("./test_feature_save.npy")
y["dev"] = np.load("./test_label_save.npy")
X["train"].shape, X["dev"].shape

((11678, 768), (1298, 768))

In [5]:
y["train"]=y["train"].squeeze()
y["dev"]=y["dev"].squeeze()
y["train"].shape, y["dev"].shape

((11678,), (1298,))

In [6]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)

model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9714946070878274

In [7]:
import sklearn.metrics as mc
predl = model.predict(X["dev"])
mc.f1_score(y["dev"], predl)

0.8370044052863437

In [8]:
fpr, tpr, thresholds = mc.roc_curve(y["dev"], pred)
mc.auc(fpr, tpr)

0.9910287410012639

In [4]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((11678, 769), (1298, 769))

In [5]:
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0]
np.all(np.linalg.eigvals(hessian) > 0)

True

In [6]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 10 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [7]:
np.all(np.linalg.eigvals(hessian) > 0)

True

In [8]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1298, 11678)

In [9]:
np.save("old_predictions.npy", pred)

In [24]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    
    #print("prediction", prediction)
    

    return X_k, y_k, prediction

In [31]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model_k.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    
  
    return change, flip, prediction,new

# exact_k

In [32]:
def exact_k(test_idx):
    for k in range(1, np.sum(y["train"])):
        _, flip,_ = new_train(k, test_idx, delta_pred)
        if flip == False:
            return k
    return None

In [33]:
"""
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

exact_ks = []
for test_idx in range(X["dev"].shape[0]):
    k =exact_k(test_idx)
    print(test_idx, k)
    exact_ks.append(k)

exact_ks=np.array(exact_ks)
np.save("exact_ks_IP.npy", exact_ks)
"""

'\n# Loop over all dev points:\nimport warnings\nwarnings.filterwarnings("ignore")\n\nexact_ks = []\nfor test_idx in range(X["dev"].shape[0]):\n    k =exact_k(test_idx)\n    print(test_idx, k)\n    exact_ks.append(k)\n\nexact_ks=np.array(exact_ks)\nnp.save("exact_ks_IP.npy", exact_ks)\n'

# Find approximate k by IF

In [34]:
def pred_change(k, scores, test_idx):
    X_k, y_k, prediction = Remove(k, delta_pred, test_idx)
    

In [35]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    
    if pred[test_idx] > 0.5:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train"].shape[0]):
        change = -np.sum(delta_pred[test_idx][top_k_index[:k]])
        #print("prediction", change)
        
        if old > 0.5 and old + change < 0.5:
            return k
        elif old < 0.5 and old + change > 0.5:
            return k
        
    return None

In [36]:
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

appro_ks = []
new_predictions = []
for test_idx in range(X["dev"].shape[0]):
    appro_k = approximate_k(test_idx)
    if appro_k != None:
        _, _, _, new_prediction = new_train(appro_k, test_idx, delta_pred)
        print(test_idx, appro_k)
        print("old", pred[test_idx], "new", new_prediction)
        print()
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
    else:
        appro_ks.append(None)
        new_predictions.append(None)



18 118
old [0.27485948] new 0.9743993646105525

20 141
old [0.22657775] new 0.9632434959894152

25 1676
old [0.88644729] new 7.513093053375219e-07

31 373
old [0.14596388] new 0.9993218328962026

53 89
old [0.27601841] new 0.9576108092522893

61 4
old [0.48484722] new 0.8624298407241414

76 83
old [0.74428304] new 0.08685835430735338

84 780
old [0.14556645] new 0.9998790966677573

88 669
old [0.85986639] new 4.45012299197562e-05

95 78
old [0.32894261] new 0.7309410052072397

101 49
old [0.65205793] new 0.3071784133459746

104 3502
old [0.8847335] new 3.863940874065676e-08

105 2582
old [0.1292337] new 0.9999983065666274

116 233
old [0.22332667] new 0.9864652646249228

123 16
old [0.57786056] new 0.42062011980368746

124 117
old [0.2808638] new 0.8491610032710978

128 79
old [0.70311474] new 0.1145858970075972

130 116
old [0.2658888] new 0.8780143126482344

154 318
old [0.19004498] new 0.9742821985528719

158 199
old [0.77736997] new 0.0471229287078373

169 130
old [0.2815472] new 0

In [39]:
appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
np.save("appro_ks_IP.npy", appro_ks)
np.save("new_predictions.npy", new_predictions)
np.save("old_predictions.npy", pred)

# Difference of change and prediction when k increasing

In [ ]:
"""
row = len(list(range(0, X["dev"].shape[0])))
col = len(list(range(1, np.sum(y["train"]))))
changes = np.zeros((row, col))
prediction = np.zeros((row, col))

import warnings
warnings.filterwarnings("ignore")
for i in range(0, X["dev"].shape[0]):
    print(i)
    for j in range(1, np.sum(y["train"]):
        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)
        
   
np.save("pred_change_app.npy", prediction)
np.save("changes.npy", changes)

difference = prediction - changes
np.save("difference_k_appk.npy", difference)
"""